# Import

In [1]:
import numpy as np
import json
from scipy.sparse import load_npz,save_npz,diags,csr_matrix
import scipy.sparse as sp
import pandas as pd
import os
from io import BytesIO
from tqdm import tqdm
from scipy.sparse.linalg import eigsh
from scipy.spatial.distance import pdist, squareform
import matplotlib.pyplot as plt
from pathlib import Path
import igraph as ig
import leidenalg as la
from matplotlib.backends.backend_pdf import PdfPages
from pypdf import PdfReader, PdfWriter
from tempfile import NamedTemporaryFile
import networkx as nx
from sklearn.neighbors import NearestNeighbors
import community as community_louvain
import pickle
from collections import defaultdict
import gc
from pympler import muppy, asizeof

# Building Multilayer Transition Matrix

In [2]:
# Load all the matrices needed
DISEASE = "BIPOLAR"
OUTPUT_DIRECTORY = f"../output/{DISEASE}/"
DGIDB_DIRECTORY = f"../../Gen_Hypergraph/output/DGIDB_{DISEASE}/"
MSIGDB_DIRECTORY = "../../Gen_Hypergraph/output/MSigDB_Full/"

## DGIDB
DGIDB_binary_matrix = load_npz(DGIDB_DIRECTORY + "hypergraph_incidence_matrix_binary.npz")
DGIDB_weighted_matrix = load_npz(DGIDB_DIRECTORY + "hypergraph_incidence_matrix_weighted.npz")
DGIDB_gene_weight_diag_matrix = load_npz(DGIDB_DIRECTORY + "gene_weight_diag_matrix.npz")
DGIDB_diag_node_degree_matrix = load_npz(DGIDB_DIRECTORY + "diag_node_degree_matrix.npz")
DGIDB_inverse_diag_edge_degree_matrix = load_npz(
    DGIDB_DIRECTORY + "inverse_diag_edge_degree_matrix.npz"
    )

## MSIGDB
MSIGDB_binary_matrix = load_npz(MSIGDB_DIRECTORY + "hypergraph_incidence_matrix_binary.npz")
MSIGDB_weighted_matrix = load_npz(MSIGDB_DIRECTORY + "hypergraph_incidence_matrix_weighted.npz")
MSIGDB_gene_weight_diag_matrix = load_npz(MSIGDB_DIRECTORY + "gene_weight_diag_matrix.npz")
MSIGDB_diag_node_degree_matrix = load_npz(MSIGDB_DIRECTORY + "diag_node_degree_matrix.npz")
MSIGDB_inverse_diag_edge_degree_matrix = load_npz(
    MSIGDB_DIRECTORY + "inverse_diag_edge_degree_matrix.npz"
    )

In [3]:
# queue = [[i] for i in range(1,25)]
# resolution = 1.0
# for T in queue:
#     path = f"../output/{DISEASE}/diffusion_dist_matrices/ddm_{T}_res-{resolution}.npy"
#     arr = np.load(path)
#     arr = arr.astype(np.float32)
#     np.save(path, arr)
#     print(f"Overwritten (float32): {path}")

In [4]:
# Useful Functions
def csr_equal_tol(A, B, atol=1e-8):
    # First check shapes and sparsity pattern
    if A.shape != B.shape or not np.array_equal(A.indptr, B.indptr) or not np.array_equal(A.indices, B.indices):
        return False
    # Compare numeric values within tolerance
    return np.allclose(A.data, B.data, atol=atol, rtol=0)

def is_symmetric(W,tol = 1e-8):
    diff = (W - W.T)
    check = np.all(np.abs(diff.data) < tol)
    return check

def degree_array(W, a=1):
    return np.asarray(W.sum(axis=a)).ravel()

def degree_diagonal_matrix(W, a=1):
    d = degree_array(W,a)
    return sp.diags(d, offsets=0, format='csr')

def symmetrically_normalize(W, a=1):
    D = np.asarray(W.sum(axis=a)).ravel()
    D_inv_sqrt = np.zeros_like(D)
    nze = D != 0
    D_inv_sqrt[nze] = 1 / np.sqrt(D[nze])

    W_sym = W.multiply(D_inv_sqrt)              # scale columns
    W_sym = W_sym.multiply(D_inv_sqrt[:, None]) # scale rows    
    return W_sym.tocsr()

def list_array_and_sparse_sizes(min_mb=1, include_locals=True, top=None):
    """
    List NumPy arrays and SciPy sparse matrices in memory, ordered by size.

    Parameters
    ----------
    min_mb : float, optional
        Minimum size (in MB) to include in the listing. Default is 1 MB.
    include_locals : bool, optional
        Whether to include local variables in addition to globals. Default True.
    top : int, optional
        If given, show only the top N largest variables.

    Notes
    -----
    - This only scans for numpy.ndarray and scipy.sparse matrices.
    - Avoids traversing other Jupyter internals (no traitlets warnings).
    """
    # snapshot current namespace to avoid "dictionary changed size" errors
    ns = dict(globals())
    if include_locals:
        ns.update(dict(locals()))

    rows = []
    for name, obj in ns.items():
        if name.startswith("_"):
            continue
        # Only handle ndarrays and sparse matrices
        if not (isinstance(obj, np.ndarray) or sp.issparse(obj)):
            continue
        try:
            size_b = asizeof.asizeof(obj)
        except Exception:
            continue
        if size_b < min_mb * 1e6:
            continue
        if isinstance(obj, np.ndarray):
            kind = f"ndarray {obj.shape} {obj.dtype}"
        elif sp.issparse(obj):
            kind = f"{type(obj).__name__} nnz={obj.nnz} shape={obj.shape}"
        else:
            kind = type(obj).__name__
        rows.append((name, kind, size_b))

    # Sort descending by size
    rows.sort(key=lambda r: r[2], reverse=True)
    total = sum(s for *_, s in rows) or 1

    # Print nicely formatted table
    print(f"{'Variable':<28}{'Type/Shape':<48}{'Size (MB)':>12}{'% of total':>12}")
    print("-" * 100)
    for i, (name, kind, size_b) in enumerate(rows):
        if top and i >= top:
            break
        print(f"{name:<28}{kind:<48}{size_b/1e6:12.2f}{100*size_b/total:12.1f}")

    print(f"\nTotal memory in arrays/sparse: {total/1e6:.2f} MB ({total/1e9:.2f} GB)")

In [5]:
# Building Adjacency Matrices
## DGIDB
H,W_v,D_v,D_e_inv = DGIDB_weighted_matrix, DGIDB_gene_weight_diag_matrix, DGIDB_diag_node_degree_matrix, DGIDB_inverse_diag_edge_degree_matrix

# Construct D_v^(-1/2)
d = (D_v @ W_v).diagonal()
d_inv_sqrt = np.zeros_like(d)
nonzero_mask = d > 0
d_inv_sqrt[nonzero_mask] = 1.0 / np.sqrt(d[nonzero_mask])
D_v_sqrt_inv = diags(d_inv_sqrt)

DGIDB_adjacency_matrix = D_v_sqrt_inv @ H @ D_e_inv @ H.T @ D_v_sqrt_inv


## MSIGDB
H,W_v,D_v,D_e_inv = MSIGDB_weighted_matrix, MSIGDB_gene_weight_diag_matrix, MSIGDB_diag_node_degree_matrix, MSIGDB_inverse_diag_edge_degree_matrix

# Construct D_v^(-1/2)
d = (D_v @ W_v).diagonal()
d_inv_sqrt = np.zeros_like(d)
nonzero_mask = d > 0
d_inv_sqrt[nonzero_mask] = 1.0 / np.sqrt(d[nonzero_mask])
D_v_sqrt_inv = diags(d_inv_sqrt)

MSIGDB_adjacency_matrix = D_v_sqrt_inv @ H @ D_e_inv @ H.T @ D_v_sqrt_inv

# Compute Degree Diagonal Matrices
DGIDB_rows_sums = degree_array(DGIDB_adjacency_matrix)
MSIGDB_rows_sums = degree_array(MSIGDB_adjacency_matrix)

In [6]:
print(is_symmetric(DGIDB_adjacency_matrix))
print(is_symmetric(MSIGDB_adjacency_matrix))

True
True


In [7]:
# Symmetric Normalization
DGIDB_adjacency_matrix = symmetrically_normalize(DGIDB_adjacency_matrix)
MSIGDB_adjacency_matrix = symmetrically_normalize(MSIGDB_adjacency_matrix)

In [8]:
print(is_symmetric(DGIDB_adjacency_matrix))
print(is_symmetric(MSIGDB_adjacency_matrix))

True
True


In [9]:
# density = DGIDB_adjacency_matrix.nnz / (DGIDB_adjacency_matrix.shape[0] * DGIDB_adjacency_matrix.shape[1])
# print("DGIDB Density:", density)
# density = MSIGDB_adjacency_matrix.nnz / (MSIGDB_adjacency_matrix.shape[0] * MSIGDB_adjacency_matrix.shape[1])
# print("MSIGDB Density:", density)

In [10]:
# Check nonzero average
DGIDB_nonzero_average = DGIDB_adjacency_matrix[DGIDB_adjacency_matrix != 0].mean()
MSIGDB_nonzero_average = MSIGDB_adjacency_matrix[MSIGDB_adjacency_matrix != 0].mean()

print("DGIDB nonzero average:",DGIDB_nonzero_average,"\nMSIGDB nonzero average:",MSIGDB_nonzero_average)

DGIDB nonzero average: 0.009289970824902828 
MSIGDB nonzero average: 9.87564623806718e-05


In [11]:
# # Density normalization version 1
# target_average = MSIGDB_nonzero_average
# DGIDB_adjacency_matrix = (target_average / DGIDB_nonzero_average) * DGIDB_adjacency_matrix
# MSIGDB_adjacency_matrix = (target_average / MSIGDB_nonzero_average) * MSIGDB_adjacency_matrix

# DGIDB_nonzero_average = DGIDB_adjacency_matrix[DGIDB_adjacency_matrix != 0].mean()
# MSIGDB_nonzero_average = MSIGDB_adjacency_matrix[MSIGDB_adjacency_matrix != 0].mean()

# print(DGIDB_nonzero_average,MSIGDB_nonzero_average)

In [12]:
# Density normalization version 2
DGIDB_mean_degree = DGIDB_rows_sums[DGIDB_rows_sums != 0].mean()
MSIGDB_mean_degree = MSIGDB_rows_sums[MSIGDB_rows_sums != 0].mean()
print(DGIDB_mean_degree,MSIGDB_mean_degree)
# print(DGIDB_rows_sums,MSIGDB_rows_sums)

DGIDB_weight = (1/DGIDB_mean_degree) / ((1/DGIDB_mean_degree)+(1/MSIGDB_mean_degree))
MSIGDB_weight = (1/MSIGDB_mean_degree) / ((1/DGIDB_mean_degree)+(1/MSIGDB_mean_degree))
geo_mean_weight = (DGIDB_weight * MSIGDB_weight)**(1/2)
print(DGIDB_weight,MSIGDB_weight)
print(geo_mean_weight)

0.5022932647447539 0.2379564756116163
0.3214543182373284 0.6785456817626716
0.46703473053286176


In [49]:
## Build interlayer coupling matrices between the two layers

# Open the JSON file and load its content into a dictionary
with open(DGIDB_DIRECTORY + "gene_to_index.json", "r") as file:
    dgidb = json.load(file)
with open(MSIGDB_DIRECTORY + "gene_to_index.json", "r") as file:
    msigdb = json.load(file)
    
# Jump probability for matching genes
w = 1

# Number of genes (assuming they are both of same size or matchable)
num_genes_dgidb = len(dgidb)
num_genes_msigdb = len(msigdb)

# Initialize the inter-layer matrix with zeros
interlayer_transition_matrix = np.zeros((num_genes_msigdb,num_genes_dgidb))
i = 0
# Build the inter-layer matrix
for gene_dgidb, idx_dgidb in dgidb.items():
    # If the gene exists in both gene-to-index mappings
    if gene_dgidb in msigdb:      
        idx_msigdb = msigdb[gene_dgidb]
        interlayer_transition_matrix[idx_msigdb,idx_dgidb] = w  # Set jump probability
        i += 1
    else:
        print(f"Gene {gene_dgidb} not found in MSIGDB mapping.")
rows_with_high_sum = np.where(interlayer_transition_matrix.sum(axis=1) > 0)[0]
print(i/len(dgidb), "of DGIDB genes have a match in MSIGDB")

Gene 927 not found in MSIGDB mapping.
Gene 11 not found in MSIGDB mapping.
Gene 724 not found in MSIGDB mapping.
Gene 360158 not found in MSIGDB mapping.
Gene 469 not found in MSIGDB mapping.
Gene 100529264 not found in MSIGDB mapping.
Gene 2609 not found in MSIGDB mapping.
Gene 447 not found in MSIGDB mapping.
Gene 453 not found in MSIGDB mapping.
Gene 459 not found in MSIGDB mapping.
Gene 2616 not found in MSIGDB mapping.
Gene 442 not found in MSIGDB mapping.
Gene 600 not found in MSIGDB mapping.
Gene 507 not found in MSIGDB mapping.
Gene 62 not found in MSIGDB mapping.
Gene 68 not found in MSIGDB mapping.
Gene 2772 not found in MSIGDB mapping.
Gene 485 not found in MSIGDB mapping.
Gene 620 not found in MSIGDB mapping.
Gene 121131 not found in MSIGDB mapping.
Gene 285834 not found in MSIGDB mapping.
Gene 6962 not found in MSIGDB mapping.
Gene 1480 not found in MSIGDB mapping.
Gene 2749 not found in MSIGDB mapping.
Gene 1624 not found in MSIGDB mapping.
Gene 63 not found in MSIGDB map

In [14]:
# # Row-Normalization
# # DGIDB
# row_sums = np.array(DGIDB_adjacency_matrix.sum(axis=1)).ravel()
# # inverse row sums (avoid division by zero)
# inv_row_sums = np.reciprocal(row_sums, where=row_sums!=0)
# # build diagonal matrix of inverses
# D_inv = sp.diags(inv_row_sums)
# DGIDB_adjacency_matrix = D_inv @ DGIDB_adjacency_matrix

# # MSIGDB
# row_sums = np.array(MSIGDB_adjacency_matrix.sum(axis=1)).ravel()
# # inverse row sums (avoid division by zero)
# inv_row_sums = np.reciprocal(row_sums, where=row_sums!=0)
# # build diagonal matrix of inverses
# D_inv = sp.diags(inv_row_sums)
# MSIGDB_adjacency_matrix = D_inv @ MSIGDB_adjacency_matrix

# # Coupling Matrix
# row_sums = interlayer_transition_matrix.sum(axis = 1, keepdims= True)
# row_sums[row_sums == 0] = 1 
# interlayer_transition_matrix = interlayer_transition_matrix / row_sums

In [15]:
# # Check for Row-stochastic
# row_sums = np.array(DGIDB_adjacency_matrix.sum(axis=1)).ravel()
# print(row_sums)
# ok = np.all(np.isclose(row_sums, 1.0)|np.isclose(row_sums, 0))
# print("Every row sums to 0 or 1?", ok)

# row_sums = np.array(MSIGDB_adjacency_matrix.sum(axis=1)).ravel()
# print(row_sums)
# ok = np.all(np.isclose(row_sums, 1.0)|np.isclose(row_sums, 0))
# print("Every row sums to 0 or 1?", ok)

In [ ]:
## Build the multilayer transition matrix
# interlayer_transition_prob = target_average
interlayer_transition_prob = 0.5

A = (1-interlayer_transition_prob) * DGIDB_weight * DGIDB_adjacency_matrix
B = interlayer_transition_prob * geo_mean_weight * interlayer_transition_matrix.T
C = interlayer_transition_prob * geo_mean_weight * interlayer_transition_matrix
D =(1-interlayer_transition_prob) * MSIGDB_weight * MSIGDB_adjacency_matrix

multilayer_transition_matrix = sp.bmat([
    [A, B],
    [C, D]
]).tocsr()

num_genes = multilayer_transition_matrix.shape[0]

del A,B,C,D, DGIDB_adjacency_matrix,MSIGDB_adjacency_matrix
gc.collect()

192

In [17]:
list_array_and_sparse_sizes() # Result should shows only MLM 

Variable                    Type/Shape                                         Size (MB)  % of total
----------------------------------------------------------------------------------------------------
multilayer_transition_matrixcsr_matrix nnz=107278214 shape=(26755, 26755)        2574.79        98.7
MSIGDB_weighted_matrix      csr_matrix nnz=563046 shape=(21981, 7411)              13.60         0.5
H                           csr_matrix nnz=563046 shape=(21981, 7411)              13.60         0.5
MSIGDB_binary_matrix        csr_matrix nnz=563046 shape=(21981, 7411)               5.72         0.2

Total memory in arrays/sparse: 2607.71 MB (2.61 GB)


In [18]:
density = multilayer_transition_matrix.nnz / (multilayer_transition_matrix.shape[0] * multilayer_transition_matrix.shape[1])
print("MTM Density:", density)

MTM Density: 0.14986548517575804


In [19]:
MTM_average = multilayer_transition_matrix[multilayer_transition_matrix != 0].mean()
print("MTM nonzero average:",MTM_average)

MTM nonzero average: 5.407776403910461e-05


In [20]:
# ## Row-normalize the multilayer transition matrix
# row_sums = np.array(multilayer_transition_matrix.sum(axis=1)).ravel()
# nonzero_rows = row_sums != 0
# inv_row_sums = np.zeros_like(row_sums)
# inv_row_sums[nonzero_rows] = 1.0 / row_sums[nonzero_rows]
# multilayer_transition_matrix = diags(inv_row_sums) @ multilayer_transition_matrix

In [21]:
row_sums = np.array(multilayer_transition_matrix.sum(axis=1)).ravel()
print(row_sums)
ok = np.all(np.isclose(row_sums, 1.0)|np.isclose(row_sums, 0))
print("Every row sums to 0 or 1?", ok)

[0.46725573 0.47856281 0.23351737 ... 0.03240383 0.02832738 0.02202548]
Every row sums to 0 or 1? False


# Computing Eigenvalues & Eigenvectors

In [22]:
num_eigenvalues = 100
num_neighbors = 50

In [23]:
# Compute top k eigenvalues by magnitude
def top_k_eigenvalues(M,k):
    vals, vecs = eigsh(M, k=k, which='LM')  # 'LM' = Largest Magnitude
    idx = np.argsort(np.abs(vals))[::-1]
    vals, vecs = vals[idx], vecs[:, idx]
    return vals, vecs

In [24]:
# Compute eigenvalues and eigenvectors
eigenvalues, eigenvectors = top_k_eigenvalues(multilayer_transition_matrix,num_eigenvalues)
print("Eigenvalues:", eigenvalues)
print("Eigenvectors shape:", eigenvectors.shape)

Eigenvalues: [0.41952326 0.41641254 0.35003217 0.33851956 0.33259475 0.33206156
 0.32372442 0.32139859 0.31509729 0.31381416 0.31296885 0.30925692
 0.30806898 0.30712403 0.3064811  0.30187841 0.29948842 0.29946554
 0.29892271 0.29763053 0.29648973 0.2950133  0.29331384 0.2917007
 0.29163329 0.29097289 0.28946062 0.2893868  0.28838826 0.2860537
 0.28510014 0.28404544 0.28330779 0.28149539 0.27966882 0.27880759
 0.27799247 0.27786739 0.27643089 0.27536967 0.27498018 0.27490854
 0.27456135 0.27451311 0.27424096 0.27342661 0.27274741 0.27149679
 0.27123133 0.2706753  0.2702983  0.26963562 0.26952014 0.26858956
 0.26839229 0.26811328 0.26794741 0.26747498 0.26728632 0.26650478
 0.26619803 0.26599131 0.26532634 0.26521145 0.265094   0.26477439
 0.26455071 0.2643002  0.26372489 0.26348133 0.26341155 0.26314818
 0.2629233  0.26281365 0.26280825 0.2626629  0.26228457 0.26209422
 0.26148524 0.26129586 0.26100384 0.2607369  0.26047373 0.26020873
 0.26012004 0.25998044 0.25984227 0.25975607 0.2595

# KNN Graph Methods & Testing (commented)

In [25]:
def stationary_distribution(W, tol=1e-12, maxit=10000):
    # power iteration for left stationary of row-stochastic W
    n = W.shape[0]
    pi = np.ones(n) / n
    for _ in range(maxit):
        pi_next = pi @ W
        if np.linalg.norm(pi_next - pi, 1) < tol:
            break
        pi = pi_next
    return pi / pi.sum()

def layer_weights_stationary(W, n):
    pi = stationary_distribution(W)
    w1 = float(pi[:n].sum()); w2 = 1.0 - w1
    return w1, w2

In [26]:
w1,w2 = layer_weights_stationary(multilayer_transition_matrix, num_genes_dgidb)

In [27]:
print(w1,w2)

0.2258699227310127 0.7741300772689873


In [28]:
def csr_diff_metrics(
    A, B,
    *,
    use_top_left_overlap=True,   # True => fast overlap by position; set False only if you align elsewhere
    compute_spectral=False,      # off by default for huge matrices
    include_support_jaccard=True,# quick structure-only difference (no weights)
    dtype=np.float32             # compact math; set to np.float64 if you need extra precision
):
    """
    Compare two very large square CSR matrices using only the overlapping block.
    Default settings are safe and fast for huge sparse inputs.

    Returns
    -------
    dict with:
      - rmse      : per-entry root-mean-square error on overlap
      - mae       : per-entry mean absolute error on overlap
      - rel_fro   : ||A-B||_F / (||A||_F + ||B||_F), in [0,1] (0 = identical)
      - cos_dist  : 1 - cosine(entrywise) on overlap (scale-insensitive)
      - supp_jacc : (optional) Jaccard on supports; distance = 1 - supp_jacc
      - spec_rel  : (optional) spectral norm ratio (expensive; off by default)
    """
    if not (sp.isspmatrix_csr(A) and sp.isspmatrix_csr(B)):
        raise TypeError("Pass CSR matrices for best performance (got types: "
                        f"{type(A)}, {type(B)}).")

    # Cast data to compact dtype without copying indices/indptr
    if A.dtype != dtype:
        A = sp.csr_matrix((A.data.astype(dtype, copy=False), A.indices, A.indptr), shape=A.shape)
    if B.dtype != dtype:
        B = sp.csr_matrix((B.data.astype(dtype, copy=False), B.indices, B.indptr), shape=B.shape)

    # Overlap by position (fast). If you need label alignment, do it upstream and keep this True.
    m = min(A.shape[0], B.shape[0]) if use_top_left_overlap else min(A.shape[0], B.shape[0])
    if m == 0:
        return dict(rmse=np.nan, mae=np.nan, rel_fro=np.nan, cos_dist=np.nan,
                    supp_jacc=np.nan if include_support_jaccard else None,
                    spec_rel=np.nan if compute_spectral else None)

    A_ = A[:m, :m]
    B_ = B[:m, :m]
    D  = A_ - B_

    m2 = float(m) * float(m)

    # Frobenius^2 via data arrays (no temporary matrices)
    froA2 = float((A_.data.astype(np.float64, copy=False)**2).sum())  # accumulate in float64 for stability
    froB2 = float((B_.data.astype(np.float64, copy=False)**2).sum())
    froD2 = float((D.data.astype(np.float64, copy=False)**2).sum())

    # Per-entry errors (size-normalized)
    rmse = np.sqrt(froD2 / m2)
    mae  = float(np.abs(D.data).sum()) / m2

    # Relative Frobenius (bounded, scale-aware)
    nA = np.sqrt(froA2); nB = np.sqrt(froB2); nD = np.sqrt(froD2)
    rel_fro = nD / (nA + nB) if (nA + nB) > 0 else 0.0

    # Cosine distance on entries (scale-insensitive)
    dotAB = float(A_.multiply(B_).sum())  # only intersects nonzeros
    cos_sim = (dotAB / (nA * nB)) if (nA > 0 and nB > 0) else 0.0
    cos_dist = 1.0 - cos_sim

    out = dict(
        rmse=float(rmse),
        mae=float(mae),
        rel_fro=float(rel_fro),
        cos_dist=float(cos_dist),
    )

    # Optional: structure-only Jaccard on supports (fast)
    if include_support_jaccard:
        SA = A_.astype(bool)
        SB = B_.astype(bool)
        inter = SA.multiply(SB).count_nonzero()
        union = SA.maximum(SB).count_nonzero()  # elementwise OR without densifying
        supp_jacc = inter / union if union else 1.0
        out["supp_jacc"] = float(supp_jacc)
        out["supp_jacc_dist"] = float(1.0 - supp_jacc)

    # Optional: spectral norm ratio (still heavy on truly huge inputs)
    if compute_spectral:
        from scipy.sparse.linalg import svds
        def spec_norm(X):
            # For very small overlaps you could densify; for big keep svds(k=1)
            if X.shape[0] <= 400:
                return float(np.linalg.norm(X.toarray(), 2))
            if X.nnz == 0:
                return 0.0
            s = svds(X, k=1, return_singular_vectors=False)
            return float(abs(s[0]))
        sA = spec_norm(A_)
        sB = spec_norm(B_)
        sD = spec_norm(D)
        denom = sA + sB
        out["spec_rel"] = (sD / denom) if denom > 0 else 0.0
    else:
        out["spec_rel"] = None

    return out


In [29]:
# Build diffusion matrix from eigenvalues and eigenvectors (double check)
def build_diffusion_dist_matrix(vals,vecs,t):
    X_t = vecs * (vals ** (t))
    D_t2 = squareform(pdist(X_t, metric='sqeuclidean'))
    return D_t2

# Build kNN
def build_kNN(diffusion_dist_matrix,k, sym_method='average'):
    # Choose sigma to be the median of pair-wise distance
    tmp = diffusion_dist_matrix.astype(np.float32, copy=True)
    np.sqrt(tmp, out=tmp)
    sigma = np.median(tmp, overwrite_input=True)

    n = diffusion_dist_matrix.shape[0]

    # Used to indicate position of nonzero value needed to record (constructing a sparse matrix for efficiency)
    rows, cols, vals = [], [], []
    for i in range(n):
        profile = np.exp(-diffusion_dist_matrix[i] / (2* (sigma**2))) 
        idx = np.argpartition(-profile, k+1)[:k+1]  # top-k+1 (includes self)
        idx = idx[idx != i]                      # drop self
        rows += [i]*k
        cols += list(idx[:k])
        vals += list(profile[idx[:k]])
    adj_mat = csr_matrix((vals, (rows, cols)), shape=(n, n))

    # Symmetrize the matrix by adding edges to one way edges and set weight to average
    if (sym_method == 'average'):
        adj_mat = (adj_mat + adj_mat.T).multiply(0.5).tocsr()

    # Symmetrize the matrix by unioning
    elif (sym_method == 'union'):
        adj_mat = adj_mat.maximum(adj_mat.T)
    elif (sym_method == 'none'):
        pass
    else:
        raise ValueError("sym_method must be 'average' or 'union'")
    
    kNN_graph = nx.from_numpy_array(adj_mat)

    
    return adj_mat,kNN_graph

In [30]:
### Build Aggregated kNN

# Load gene-to-index mappings
with open(DGIDB_DIRECTORY+ "gene_to_index.json", "r") as f:
    DGIDB_gene_to_index = json.load(f)
with open(MSIGDB_DIRECTORY + "gene_to_index.json", "r") as f:
    MSIGDB_gene_to_index = json.load(f)
# Reverse the dictionary
DGIDB_index_to_gene = {index: gene for gene, index in DGIDB_gene_to_index.items()}


### Prerequisite: interlayer_transition_matrix, num_genes_dgidb, num_genes_msigdb
def build_aggregated_kNN(diffusion_dist_matrix,
                         k,
                         n1 = num_genes_dgidb,
                         n2 = num_genes_msigdb,
                         sym_method='average'):
    w1,w2 = layer_weights_stationary(multilayer_transition_matrix, num_genes_dgidb)
    
    similarity_matrix = build_kNN(diffusion_dist_matrix,k)[0]
    
    DGIDB_sim_matrix = similarity_matrix[:n1, :n1]
    MSIGDB_sim_matrix = similarity_matrix[n1:, n1:]

    DGIDB_index_map = {}
    gene_to_index_additional = {}
    coupling_matrix = interlayer_transition_matrix.T
    num_additional_rows = 0
    index = 0
    for row in coupling_matrix:
        if(np.all(row == 0)):
            DGIDB_index_map[index] = n2 + num_additional_rows 
            gene_to_index_additional[DGIDB_index_to_gene[index]] = n2 + num_additional_rows
            num_additional_rows += 1
        else:
            l = np.nonzero(row)[0]
            if (len(l) > 1):
                print("ERROR")
            else:
                msigdb_pos = l[0]
                DGIDB_index_map[index] = msigdb_pos
        index += 1

    zero_block = sp.csr_matrix((num_additional_rows, num_additional_rows), dtype=MSIGDB_sim_matrix.dtype)
    asm = sp.block_diag((MSIGDB_sim_matrix, zero_block), format="csr")

    # Build and save gene_to_index_distinct mapping
    gene_to_index_distinct = MSIGDB_gene_to_index | gene_to_index_additional

    save_path = OUTPUT_DIRECTORY + "gene_to_index_distinct.json"
    if not os.path.exists(save_path):
        with open(save_path, 'w') as pathway_file:
            json.dump(gene_to_index_distinct, pathway_file, indent=4)
        print(f"Mappings saved to {save_path}")  
    ### Aggregation

    # rows, cols = DGIDB_sim_matrix.nonzero()
    # for i, j in zip(rows, cols):
    #     msig_i,msig_j = DGIDB_index_map[i], DGIDB_index_map[j]
    #     if (asm[msig_i,msig_j] == 0):
    #         asm[msig_i, msig_j] = DGIDB_sim_matrix[i, j]
    #     else:
    #         dgidb_sim_val = DGIDB_sim_matrix[i, j]
    #         msigdb_sim_val = asm[msig_i, msig_j]
    #         asm[msig_i, msig_j] = (w1 * dgidb_sim_val + w2 * msigdb_sim_val)   

    M = asm.tocsr()

    # 1) Get COO to access row/col vectors
    DGIDB = DGIDB_sim_matrix.tocoo(copy=False)
    row = DGIDB.row.astype(np.int64, copy=False)
    col = DGIDB.col.astype(np.int64, copy=False)

    # 2) Turn the mapping into a 1-D integer array so we can do map_arr[row], map_arr[col]
    N = DGIDB_sim_matrix.shape[0]

    if isinstance(DGIDB_index_map, dict):
        # Fill with -1 to detect missing keys
        map_arr = np.full(N, -1, dtype=np.int64)
        for k, v in DGIDB_index_map.items():
            if 0 <= k < N:
                map_arr[k] = int(v)
        if (map_arr < 0).any():
            missing = np.flatnonzero(map_arr < 0)
            raise ValueError(f"DGIDB_index_map is missing {missing.size} DGIDB indices; "
                            f"first few missing: {missing[:5].tolist()}")
    else:
        # list/ndarray case
        map_arr = np.asarray(DGIDB_index_map, dtype=np.int64)
        if map_arr.ndim != 1 or map_arr.shape[0] != N:
            raise ValueError(f"DGIDB_index_map must be length {N} 1-D; got shape {map_arr.shape}")

    # 3) Vectorized remap of DGIDB → MSIGDB indices
    msig_r = map_arr[row]
    msig_c = map_arr[col]

    # 4) Build remapped DGIDB matrix D in MSIGDB space (CSR)
    D = sp.csr_matrix((DGIDB.data, (msig_r, msig_c)), shape=M.shape)

    # 5) Vectorized aggregation (exactly matches your if/else logic)
    Mb = M.copy(); Mb.data[:] = 1   # support(M)
    Db = D.copy(); Db.data[:] = 1   # support(D)

    M_keep = M - M.multiply(Db)                             # keep M where D==0
    
    D_only = D - D.multiply(Mb) # take D where M==0
    
    overlap = w1 * D.multiply(Mb) + w2 * M.multiply(Db)     # where both nonzero

    # aggregated = (M_keep + D_only + overlap).tocsr()
    aggregated = (M_keep).tocsr() #WRONG NEED TO BE CHANGED
    aggregated.eliminate_zeros()

    # Result
    asm = aggregated

    if (sym_method not in ['average','union','none']):
        raise ValueError("sym_method must be either 'average' or 'union'")
    elif (sym_method == 'none'):
        # No symmetrization
        pass
    elif (sym_method == 'average'):
        # Symmetrize the matrix by adding edges to one way edges and set weight to average
        asm = (asm + asm.T).multiply(0.5).tocsr()
    elif (sym_method == 'union'):
        # Symmetrize the matrix by unioning
        asm = asm.maximum(asm.T)

    graph = nx.from_scipy_sparse_array(asm)
    
    return asm, graph
        

In [31]:
# ddm = build_diffusion_dist_matrix(eigenvalues, eigenvectors, [2])

In [32]:
# knn_orig,_ = build_kNN(ddm,num_neighbors)
# knn_new,_,knn_sus = build_aggregated_kNN(ddm,num_neighbors)
# csr_equal_tol(knn_orig,knn_sus)
# print(is_symmetric(knn_orig),is_symmetric(knn_new))
# print(knn_orig)
# print(knn_new)

In [33]:
# print(csr_diff_metrics(knn_orig, knn_new,use_top_left_overlap=True))
# print(knn_orig.shape, knn_new.shape)
# print(csr_diff_metrics(knn_orig[num_genes_dgidb:, num_genes_dgidb:], knn_new,use_top_left_overlap=True))
# print(knn_orig[num_genes_dgidb:, num_genes_dgidb:].shape, knn_new.shape)

In [34]:
# density_orig = knn_orig.nnz / (knn_orig.shape[0] * knn_orig.shape[1])
# print(density_orig)
# density_new = knn_new.nnz / (knn_new.shape[0] * knn_new.shape[1])
# print(density_new)

In [35]:
# print(knn_orig.nnz, knn_new.nnz)
# print(knn_orig.data.mean(), knn_new.data.mean())

In [36]:
# del ddm, knn_orig, knn_sus, knn_new
# gc.collect()

# Clustering Methods

In [41]:
# # Louvain
# def louvain_from_adj(A, resolution=1.0, random_state=0, keep_lcc=False):
#     """
#     A: symmetric, non-negative scipy.sparse adjacency (CSR preferred).
#     Returns: labels (np.ndarray of length n), graph G (NetworkX), node_order
#     """
#     # if not sp.isspmatrix(A):  # allow dense but convert
#     #     A = sp.csr_matrix(A)
#     # # Optional: ensure symmetry numerically
#     # if (A - A.T).nnz != 0:
#     #     raise ValueError("Adjacency must be symmetric. Symmetrize first.")

#     # Build graph
#     # networkx >=3.0: from_scipy_sparse_array; older: from_scipy_sparse_matrix
#     G = nx.from_scipy_sparse_array(A, edge_attribute='weight')  # undirected by default

#     if keep_lcc:
#         # keep only the largest connected component if you prefer
#         largest_cc = max(nx.connected_components(G), key=len)
#         G = G.subgraph(largest_cc).copy()

#     # Run Louvain
#     part = community_louvain.best_partition(
#         G, weight='weight', resolution=resolution, random_state=random_state
#     )
#     node_order = sorted(G.nodes())
#     labels = np.array([part[i] for i in node_order], dtype=int)
#     return labels, G, node_order

# def DDBC(MTM,num_eigenvalues,num_neighbors,resolution,T):
#     vals,vecs = top_k_eigenvalues(M = MTM, k = num_eigenvalues)

#     diffusion_dist_matrix = np.empty((num_genes,num_genes))
#     for t in T:
#         np.add(diffusion_dist_matrix, 
#                build_diffusion_dist_matrix(vals,vecs,t,num_eigenvalues), 
#                out=diffusion_dist_matrix)
#     diffusion_dist_matrix = diffusion_dist_matrix / len(T)
    
#     kNN_adjacency_matrix = build_kNN(diffusion_dist_matrix,num_neighbors)

#     # # Symmetrize the matrix by adding edges to one way edges and set weight to average
#     # kNN_adjacency_matrix = (kNN_adjacency_matrix + kNN_adjacency_matrix.T).multiply(0.5).tocsr()

#     # Symmetrize the matrix by unioning
#     kNN_adjacency_matrix = kNN_adjacency_matrix.maximum(kNN_adjacency_matrix.T)

#     return louvain_from_adj(kNN_adjacency_matrix, resolution = resolution, keep_lcc = False) 

In [42]:
def leiden_from_knn_adjacency(
    A,
    *,
    method="modularity",      # "modularity" | "rb" | "cpm"
    resolution=1.0,           # used by "rb" and "cpm"
    n_iterations=-1,          # -1 => until no improvement
    seed=42,
    use_weights=True
):
    """
    Run Leiden on a symmetric, undirected (weighted) kNN adjacency (SciPy sparse).

    Returns
    -------
    labels : np.ndarray[int]
        Community ID per node (0..k-1)
    quality : float
        Objective value (modularity / RB / CPM depending on 'method')
    """
    if not sp.issparse(A):
        raise TypeError("A must be a SciPy sparse matrix.")
    # Normalize format & dtype
    A = A.tocsr().astype(np.float32, copy=False)

    # Clean diagonal and robustly symmetrize
    A.setdiag(0.0)
    A.eliminate_zeros()
    A = A.maximum(A.T)  # keep max weight per undirected edge

    # Build igraph from upper triangle (each undirected edge once)
    U = sp.triu(A, k=1, format="coo")
    n = A.shape[0]
    g = ig.Graph(n=n, edges=list(zip(U.row.tolist(), U.col.tolist())), directed=False)
    wname = None
    if use_weights:
        g.es["weight"] = U.data.tolist()
        wname = "weight"

    # Pick partition class + kwargs
    m = method.lower()
    if m == "modularity":
        part_cls = la.ModularityVertexPartition
        kwargs = dict(weights=wname)
    elif m == "rb":
        part_cls = la.RBConfigurationVertexPartition
        kwargs = dict(weights=wname, resolution_parameter=resolution)
    elif m == "cpm":
        part_cls = la.CPMVertexPartition
        kwargs = dict(weights=wname, resolution_parameter=resolution)
    else:
        raise ValueError("method must be 'modularity', 'rb', or 'cpm'")

    # Run Leiden
    part = la.find_partition(
        g,
        part_cls,
        n_iterations=n_iterations,
        seed=seed,
        **kwargs
    )

    labels = np.array(part.membership, dtype=np.int32)
    communities = [list(c) for c in part]
    return labels, float(part.quality()), communities

In [60]:
def DDBC(vals,
         vecs,
         num_neighbors,
         resolution,
         T,
         leiden_method = "modularity",
         ddm = None):
    if ddm is None:
        diffusion_dist_matrix = np.empty((num_genes,num_genes))
        for t in T:
            np.add(diffusion_dist_matrix, 
                build_diffusion_dist_matrix(vals,vecs,t), 
                out=diffusion_dist_matrix)
        diffusion_dist_matrix = diffusion_dist_matrix / len(T)
        np.save(f"../output/{DISEASE}/diffusion_dist_matrices/ddm_{T}_res-{resolution}_itp-{interlayer_transition_prob}.npy",diffusion_dist_matrix)
    else:
        diffusion_dist_matrix = ddm
    # kNN_adjacency_matrix, kNN_graph = build_kNN(diffusion_dist_matrix,num_neighbors)
    kNN_adjacency_matrix, kNN_graph = build_aggregated_kNN(diffusion_dist_matrix,num_neighbors)

    return (*leiden_from_knn_adjacency(kNN_adjacency_matrix,method=leiden_method,resolution=resolution,n_iterations=-1,seed=42), kNN_graph)

In [44]:
def community_report_onepage(labels, score, out_pdf="leiden_community_report.pdf", *,
                             extra_text=None, bins="auto", title="Community Sizes", time_steps = "N/A"):
    """
    Create ONE PDF page (top: text summary, bottom: histogram).
    If out_pdf exists: append this page. Else: create it.

    Requires: matplotlib, pypdf  (pip install pypdf)
    """
    # ---- inputs ----
    labels = np.asarray(labels)
    if labels.ndim != 1:
        raise ValueError("labels must be a 1-D array of community ids")

    # ---- stats ----
    sizes = np.bincount(labels.astype(np.int64, copy=False))
    sizes_sorted = np.sort(sizes)
    n, k = sizes.sum(), sizes.size

    lines = [
        "Leiden Partition Summary",
        "========================",
        f"Nodes (n):           {n:,}",
        f"Time steps:          {time_steps}",
        f"Communities (k):     {k:,}",
        f"Size (min):          {int(sizes_sorted[0]) if k else 0:,}",
        f"Size (median):       {float(np.median(sizes_sorted)) if k else 0.0:.3f}",
        f"Size (mean):         {float(sizes_sorted.mean()) if k else 0.0:.6f}",
        f"Size (max):          {int(sizes_sorted[-1]) if k else 0:,}",
        f"Score:               {score}",
        "",
        "Top 10 largest communities (id: size):",
    ]
    for cid in np.argsort(-sizes)[:min(10, k)]:
        lines.append(f"  {int(cid):5d}: {int(sizes[cid]):,}")
    if extra_text:
        lines += ["", "Extra:", *([extra_text] if isinstance(extra_text, str) else list(extra_text))]
    summary_text = "\n".join(lines)

    # ---- draw single-page figure ----
    fig = plt.figure(figsize=(8.5, 11), dpi=150)          # US Letter, higher DPI
    ax_text = fig.add_axes([0.06, 0.55, 0.88, 0.40])      # [left, bottom, width, height]
    ax_text.axis("off")
    ax_text.text(0.0, 1.0, summary_text, va="top", ha="left", fontsize=11, family="monospace")

    ax_hist = fig.add_axes([0.10, 0.08, 0.80, 0.38])
    ax_hist.hist(sizes, bins=bins)
    ax_hist.set_xlabel("Community size")
    ax_hist.set_ylabel("Count of communities")
    ax_hist.set_title(f"Distribution of {title}")

    # ---- write this page to a temp PDF on disk ----
    with NamedTemporaryFile(delete=False, suffix=".pdf") as tmpf:
        tmp_page = Path(tmpf.name)
    with PdfPages(tmp_page) as pdf:
        pdf.savefig(fig)
    plt.close(fig)

    # ---- append/create using PdfReader/PdfWriter (robust across versions) ----
    try:
        from pypdf import PdfReader, PdfWriter
    except Exception as e:
        try: os.remove(tmp_page)
        except: pass
        raise RuntimeError("Please install 'pypdf' (e.g., `pip install pypdf`).") from e

    out_pdf = Path(out_pdf)
    tmp_out = out_pdf.with_suffix(out_pdf.suffix + ".tmp")

    writer = PdfWriter()

    # if existing, copy old pages first
    if out_pdf.exists():
        with open(out_pdf, "rb") as f_exist:
            reader = PdfReader(f_exist)
            for p in reader.pages:
                writer.add_page(p)

    # add the new single page
    with open(tmp_page, "rb") as f_new:
        reader_new = PdfReader(f_new)
        for p in reader_new.pages:
            writer.add_page(p)

    # atomic write
    with open(tmp_out, "wb") as f_out:
        writer.write(f_out)
    os.replace(tmp_out, out_pdf)

    # cleanup
    try: os.remove(tmp_page)
    except: pass

    return out_pdf


# Run DDBC 1 - 24

In [ ]:
# Leiden Clustering
pdf_path = f"../output/{DISEASE}/leiden_report.pdf"
method = "modularity"
resolution = 1.0
num_bins = 100
write = True
queue = [[i] for i in range(1,25)]
label_list = []
score_list = []
graph_list = []
communities_list = []

In [ ]:
# # Load variables:
# DATA_DIRECTORY = OUTPUT_DIRECTORY + "leiden_result_variables_temp"
# with open(f"{DATA_DIRECTORY}/label.pkl", "rb") as f:
#     label_list = pickle.load(f)
# with open(f"{DATA_DIRECTORY}/score.pkl", "rb") as f:
#     score_list = pickle.load(f)
# with open(f"{DATA_DIRECTORY}/graph.pkl", "rb") as f:
#     graph_list = pickle.load(f)
# with open(f"{DATA_DIRECTORY}/communities.pkl", "rb") as f:
#     communities_list = pickle.load(f)

In [ ]:
for T in queue:
    ddm = np.load(f"../output/{DISEASE}/diffusion_dist_matrices/ddm_{T}_res-{resolution}.npy")
    labels, score, communities, graph = DDBC(eigenvalues, eigenvectors, num_neighbors, resolution, T, leiden_method = method, ddm = ddm)
    if (write):
        path = community_report_onepage(labels, score, out_pdf=pdf_path,
                                        extra_text=[f"method={method}", f"resolution={resolution}"], 
                                        bins=num_bins,time_steps = T)
        print("Wrote:", path)
    label_list.append(labels)
    score_list.append(score) 
    graph_list.append(graph)
    communities_list.append(communities)

In [ ]:
# # Save resulting variables to save time
# DATA_DIRECTORY = OUTPUT_DIRECTORY + "leiden_result_variables_temp"
# with open(f"{DATA_DIRECTORY}/label.pkl", "wb") as f:
#     pickle.dump(label_list, f)
# with open(f"{DATA_DIRECTORY}/score.pkl", "wb") as f:
#     pickle.dump(score_list, f)
# with open(f"{DATA_DIRECTORY}/graph.pkl", "wb") as f:
#     pickle.dump(graph_list, f)
# with open(f"{DATA_DIRECTORY}/communities.pkl", "wb") as f:
#     pickle.dump(communities_list, f)

# Visualization

In [ ]:
def community_central_genes(G, community_nodes, weight="weight", top_n=20):
    C = set(community_nodes)
    H = G.subgraph(C).copy()                       # induced subgraph
    # within-community (weighted) degree
    k = {u: H.degree(u, weight=weight) for u in H}
    ks = np.array(list(k.values()), dtype=float)
    mu, sigma = ks.mean(), ks.std() if ks.std() > 0 else 1.0
    Z = {u: (k[u] - mu)/sigma for u in H}          # within-module degree z-score

    # rank by z
    ranked = sorted(H.nodes(), key=lambda u: (Z[u]), reverse=True)
    return {u : Z[u] for u in ranked[:top_n]}

def weighted_jaccard(scoresA, scoresB):
    """
    Compute Weighted Jaccard similarity between two communities
    based on gene importance scores.
    
    Parameters:
        scoresA, scoresB : dict
            {gene: importance_score}
            Scores can be any nonnegative values (e.g., PageRank, Z-score).
    Returns:
        float
            Weighted Jaccard similarity in [0, 1].
    """
    genes = set(scoresA) | set(scoresB)
    if not genes:
        return 0.0
    num = sum(min(scoresA.get(g, 0.0), scoresB.get(g, 0.0)) for g in genes)
    den = sum(max(scoresA.get(g, 0.0), scoresB.get(g, 0.0)) for g in genes)
    return num / den if den > 0 else 0.0

def weighted_overlap_coefficient(dictA, dictB):
    """
    Weighted Szymkiewicz–Simpson (Overlap) coefficient ∈ [0,1].
    """
    if not dictA or not dictB:
        return 0.0

    common = set(dictA) & set(dictB)
    inter_sum = sum(min(dictA[v], dictB[v]) for v in common)

    sumA = sum(max(0, w) for w in dictA.values())
    sumB = sum(max(0, w) for w in dictB.values())
    denom = min(sumA, sumB)

    return inter_sum / denom if denom > 0 else 0.0




In [ ]:
def wjacc_edges_builder(g_list,c_list,score_cap = 0,com_size_cap = 0):
    result = []
    for i in range(len(g_list)-1):
        prev_G = g_list[i]
        curr_G = g_list[i+1]
        prev_com = c_list[i]
        curr_com = c_list[i+1]
        prev_z = []
        curr_z = []
        
        for com in prev_com:
            if (len(com) > com_size_cap):
                prev_z.append(community_central_genes(prev_G,com))
        for com in curr_com:
            if (len(com) > com_size_cap):
                curr_z.append(community_central_genes(curr_G,com))

        for j in range(len(prev_z)):
            for k in range(len(curr_z)):
                jaccard_score = weighted_jaccard(prev_z[j],curr_z[k])
                if (jaccard_score >= score_cap):
                    result.append((queue[i][0],j,k,jaccard_score))
    
    return result

def community_sizes(labels):
    """Return dict[label] -> number of nodes in that community."""
    lab = np.asarray(labels)
    uniq, cnt = np.unique(lab, return_counts=True)
    return {int(c): int(n) for c, n in zip(uniq, cnt)}

def topk_at_t1(labels_t1, k=10):
    """Return list of top-k community labels at t=1 by size."""
    sizes = community_sizes(labels_t1)
    return [c for c, _ in sorted(sizes.items(), key=lambda x: x[1], reverse=True)[:k]], sizes

# ---------- Jaccard lookup ----------

def build_edge_lookup(wjacc_edges):
    """
    Build a dict mapping (t, comm_t) -> list of (comm_t+1, jaccard_score).
    Each wjacc_edges element = (t, comm_t, comm_tplus1, jaccard_score)
    """
    out = defaultdict(list)
    for t, c1, c2, s in wjacc_edges:
        out[(int(t), int(c1))].append((int(c2), float(s)))
    return out

# ---------- Tracking ----------

def track_paths(seeds, wj_lookup, ts):
    """Track each seed community forward by max Jaccard each step."""
    print(ts)
    t0 = ts[0]
    paths = {s: [(t0, s)] for s in seeds}
    edges = []
    for s in seeds:
        cur = s
        for i in range(len(ts) - 1):
            t = ts[i]
            tp1 = ts[i+1]
            cand = wj_lookup.get((t, cur), [])
            if not cand:
                print(f"Seed {s}, {t} to {tp1} has no outgoing edges. Termniated.")
                paths[s].append((tp1, None))
                cur = None
                break
            nxt, score = max(cand, key=lambda x: x[1])
            edges.append(((t, cur), (tp1, nxt), score))
            paths[s].append((tp1, nxt))
            cur = nxt
    return paths, edges

# MY VERSION
# def track_paths(labels_by_t, seeds, wj_lookup, ts,score_cap = 0.1):
#     """Track each seed community forward by max Jaccard each step."""
#     t0 = ts[0]
#     paths = {s: [(t0, s)] for s in seeds}
#     edges = []
#     curs = []
#     curs_next = []
#     for s in seeds:
#         curs_next.append(s)
#         for i in range(len(ts) - 1):
#             curs = curs_next
#             curs_next = []
#             for cur in curs:
#                 t = ts[i]
#                 tp1 = ts[i+1]
#                 cand = wj_lookup.get((t, cur), [])
#                 if not cand:
#                     if (t == t0):
#                         print(f"Community {cur} has no outgoing edges")
#                     paths[s].append((tp1, None))
#                     cur = None
#                     continue
                
                
#                 for nxt,score in cand:
#                     if (score >= score_cap):
#                         edges.append(((t, cur), (tp1, nxt), score))
#                         paths[s].append((tp1, nxt))
#                         curs_next.append(nxt)

#     return paths, edges

# ---------- Get node sizes ----------

def node_sizes_for_paths(labels_by_t, paths):
    """Return dict[(t, comm)] -> size (number of vertices)."""
    out = {}
    for s, seq in paths.items():
        for t, c in seq:
            if c is None: continue
            sizes = community_sizes(labels_by_t[t])
            out[(t, c)] = sizes.get(c, 0)
    return out

# ---------- Draw layered network ----------

def draw_layered_paths(paths, edge_list, node_sizes, ts,
                       node_size_scale=700.0, edge_width_scale=8.0,
                       seed_gap=1.0, col_gap=3.0, score_cap = 0.5, title=None, save_path = None):
    seeds = list(paths.keys())
    G = nx.DiGraph()

    # Add nodes
    for s in seeds:
        for t, c in paths[s]:
            if c is None: continue
            G.add_node((s, t, c), seed=s, t=t, comm=c, size=node_sizes.get((t, c), 0))
    # Add edges
    # Set your inclusion threshold (cap)
    # include all edges with Jaccard >= 0.5

    for (t, c1), (tp, c2), w in edge_list:
        if w < score_cap:
            continue
        for s in seeds:
            seq = paths[s]
            for i in range(len(seq) - 1):
                if seq[i] == (t, c1) and seq[i+1] == (tp, c2):
                    G.add_edge((s, t, c1), (s, tp, c2), jacc=w)

    # Layout positions
    pos = {}
    nodes_by_t = defaultdict(list)
    for n in G.nodes:
        t = G.nodes[n]['t']
        nodes_by_t[t].append(n)

    # Sort nodes in each column by community size (largest first)
    pos = {}
    for t in ts:
        column_nodes = nodes_by_t.get(t, [])
        # sort by size descending
        column_nodes.sort(key=lambda n: G.nodes[n].get('size', G.nodes[n].get('size_w', 0)), reverse=True)
        for i, n in enumerate(column_nodes):
            pos[n] = (ts.index(t) * col_gap, -i * seed_gap)

    # Scale sizes and widths
    if len(G) == 0:
        raise ValueError("No nodes to draw — check your data.")
    vals = np.array([G.nodes[n]['size'] for n in G.nodes], dtype=float)
    vmin, vmax = vals.min(), vals.max()
    sizes = []
    for n in G.nodes:
        w = G.nodes[n]['size']
        a = (w - vmin) / (vmax - vmin) if vmax > vmin else 1.0
        sizes.append((0.3 + 0.7*a) * node_size_scale)
    widths = [max(0.5, d['jacc'] * edge_width_scale) for _, _, d in G.edges(data=True)]

    # Draw
    plt.figure(figsize=(max(10, len(ts)*1.4), max(6, len(seeds)*0.55 + 2)))
    nx.draw_networkx_nodes(G, pos, node_size=sizes)
    nx.draw_networkx_edges(G, pos, width=widths, arrows=False)

    edge_labels = {(u, v): f"{d['jacc']:.2f}" for u, v, d in G.edges(data=True)}
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=8)
    
    plt.xticks([i * col_gap for i in range(len(ts))], [str(t) for t in ts])
    plt.yticks([])
    plt.title(title or f"Top {len(seeds)} communities from t={ts[0]} → t={ts[-1]} (edge ∝ Jaccard)")
    plt.tight_layout()
    if (save_path != None):
        plt.savefig(save_path, dpi=300, bbox_inches="tight")
    plt.show()

# ---------- Main wrapper ----------

def visualize_topk_from_edges(labels_by_t, wjacc_edges,
                              t_start=1, t_end=16, top_k=10,
                              node_size_scale=700.0, score_cap = 0.5,
                              edge_width_scale=8.0, title=None, save_path = None):
    ts = sorted(labels_by_t.keys())
    if t_start not in labels_by_t:
        raise ValueError(f"Missing labels for t={t_start}")

    # Top-k seeds from t_start
    seeds, _ = topk_at_t1(labels_by_t[t_start], k=top_k)

    # Track forward using provided Jaccard edges
    wj_lookup = build_edge_lookup(wjacc_edges)
    paths, edges = track_paths(seeds, wj_lookup, ts)

    # Compute community sizes (unweighted)
    node_sizes = node_sizes_for_paths(labels_by_t, paths)

    # Draw the layered visualization
    print(ts)
    draw_layered_paths(paths, edges, node_sizes, ts,
                       node_size_scale=node_size_scale,
                       edge_width_scale=edge_width_scale,
                       title=title or f"Top-{top_k} from t={t_start} → t={t_end}", score_cap = score_cap,
                       save_path = save_path)

    return paths, edges


In [ ]:
# # Load variables:
# DATA_DIRECTORY = OUTPUT_DIRECTORY + "leiden_result_variables_temp"
# with open(f"{DATA_DIRECTORY}/label.pkl", "rb") as f:
#     label_list = pickle.load(f)
# with open(f"{DATA_DIRECTORY}/score.pkl", "rb") as f:
#     score_list = pickle.load(f)
# with open(f"{DATA_DIRECTORY}/graph.pkl", "rb") as f:
#     graph_list = pickle.load(f)
# with open(f"{DATA_DIRECTORY}/communities.pkl", "rb") as f:
#     communities_list = pickle.load(f)

In [ ]:
print(len(graph_list))
print(len(communities_list))

In [ ]:
labels_dict = {queue[i][0]: label_list[i] for i in range(len(queue))}
wjacc = wjacc_edges_builder(graph_list,communities_list)

In [ ]:
print(labels_dict)
print(len(labels_dict))

In [ ]:
print(wjacc)
print(len(wjacc))
save_path = f"../output/{DISEASE}/leiden_tracking_results/leiden_tracking.png"
# save_path = None
_, edges = visualize_topk_from_edges(labels_dict, wjacc, 1,24,top_k = 20,score_cap = 0.03,save_path=save_path)

In [ ]:
print(score_list)

In [ ]:
queue = [[i] for i in range(1,25)]
x = [T[0] for T in queue]
y1 = score_list
y2 = [len(communities_list[i]) for i in range(len(communities_list))]

# Create a figure with 2 subplots (1 row, 2 columns)
fig, axes = plt.subplots(1, 2, figsize=(10, 4))

# Left graph
axes[0].plot(x, y1, color='r')
axes[0].set_title("quality score")
axes[0].set_xlabel("x")
axes[0].set_ylabel("quality_score")
axes[0].grid(True)

# Right graph
axes[1].plot(x, y2, color='b')
axes[1].set_title("community size")
axes[1].set_xlabel("x")
axes[1].set_ylabel("community size")
axes[1].grid(True)

# Adjust spacing
plt.tight_layout()
plt.show()

In [ ]:
mixed_time = 1 / (1-eigenvalues[1])
print(f"Mixed Time: {mixed_time}")

# Final Average DDBC

In [45]:
# Final Average Clustering
average_t = [2,4,6,8,10,12]
pdf_path = f"../output/{DISEASE}/leiden_report.pdf"
method = "modularity"
num_neighbors = 100
resolution = 1.0
label_list = []
score_list = []
graph_list = []
communities_list = []

In [46]:
# ddm_list = []
# for i in average_t:
#     ddm = np.load(f"../output/{DISEASE}/diffusion_dist_matrices/ddm_[{i}]_res-{resolution}.npy")
#     ddm_list.append(ddm)

In [58]:
ddm = np.load(f"../output/{DISEASE}/diffusion_dist_matrices/ddm_{average_t}_res-{resolution}.npy")

In [61]:
labels, score, communities, graph = DDBC(eigenvalues, eigenvectors, num_neighbors, resolution, average_t, leiden_method = method)
path = community_report_onepage(labels, score, out_pdf=pdf_path,
                                extra_text=[f"method={method}", f"resolution={resolution}"], 
                                bins=100,time_steps = average_t)
print("Wrote:", path)

Wrote: ..\output\BIPOLAR\leiden_report.pdf


In [63]:
del ddm
gc.collect()

154765

In [68]:
list_array_and_sparse_sizes()

Variable                    Type/Shape                                         Size (MB)  % of total
----------------------------------------------------------------------------------------------------
interlayer_transition_matrixndarray (21981, 4774) float64                         839.50        91.7
eigenvectors                ndarray (26755, 100) float64                           42.81         4.7
MSIGDB_weighted_matrix      csr_matrix nnz=563046 shape=(21981, 7411)              13.60         1.5
H                           csr_matrix nnz=563046 shape=(21981, 7411)              13.60         1.5
MSIGDB_binary_matrix        csr_matrix nnz=563046 shape=(21981, 7411)               5.72         0.6

Total memory in arrays/sparse: 915.23 MB (0.92 GB)


In [66]:
print(communities)
DATA_DIRECTORY = OUTPUT_DIRECTORY + "leiden_results"
with open(f"{DATA_DIRECTORY}/result_communities.pkl", "wb") as f:
    pickle.dump(communities, f)
with open(f"{DATA_DIRECTORY}/result_graph.pkl", "wb") as f:
    pickle.dump(graph, f)    

[[11, 12, 21, 39, 43, 47, 50, 52, 54, 55, 58, 59, 62, 67, 68, 82, 83, 94, 101, 102, 112, 113, 114, 119, 120, 122, 129, 132, 138, 142, 144, 145, 146, 148, 153, 154, 159, 160, 161, 164, 188, 195, 202, 203, 204, 205, 206, 207, 208, 212, 213, 217, 218, 220, 221, 222, 224, 234, 235, 237, 238, 239, 240, 241, 250, 255, 258, 259, 263, 265, 267, 271, 272, 274, 278, 279, 281, 284, 287, 288, 289, 291, 292, 295, 296, 297, 299, 300, 303, 307, 308, 320, 328, 337, 349, 389, 390, 398, 399, 400, 401, 402, 403, 404, 405, 407, 409, 411, 412, 413, 414, 415, 416, 421, 424, 426, 427, 440, 445, 447, 448, 449, 451, 452, 458, 459, 460, 462, 463, 464, 465, 467, 468, 469, 470, 472, 475, 477, 478, 479, 482, 484, 486, 487, 491, 494, 495, 498, 499, 501, 505, 518, 522, 530, 535, 538, 539, 540, 541, 562, 575, 579, 580, 581, 583, 585, 590, 594, 595, 596, 602, 603, 605, 606, 608, 610, 611, 614, 618, 619, 620, 621, 622, 625, 626, 627, 629, 634, 641, 642, 643, 645, 647, 648, 649, 652, 655, 659, 661, 664, 669, 674, 675, 6